# Data Epoching, Behavioral Data Integration, Graphing Individual ERP Notebook

Create EEG epochs from event markers: Segmenting the continuous EEG data based on event markers.
Load the behavioral data into a dataframe: Importing and preprocessing behavioral data to align with the EEG data.

In [4]:
# First, load the necessary packages
import os
import numpy as np
import glob as glob
import mne
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats as spst
import pandas as pd
import sys
import time
import joblib

# Determine which computer you're running on, setting the /mindstore/ezzyatlab 
# server location accordingly
import socket
if socket.gethostname() == 'Youssefs-iMac.local': # home office
    server_folder = '/Volumes/ezzyatlab/'
elif socket.gethostname() == 'Youssefs-MacBook-Air.local': # laptop
    server_folder = '/Volumes/ezzyatlab/'
elif socket.gethostname() == 'yezzyat-21': # lab office
    server_folder = '/Volumes/ezzyatlab/'
else:
    server_folder = '/Volumes/ezzyatlab/'

# Using the server location, load a library of lab code to be used
# for post-processing

sys.path.append(server_folder + 'labutils/') 
sys.path.append(server_folder + 'labutils/scalpeeg/') 

#exp_folder = server_folder + 'experiments/NEvent/'

#participants = pd.read_csv(exp_folder + 'participants.tsv',
#                           delimiter='\t')

#print(f'\nMNE-Python Version: {mne.__version__}\n')

# Set the subject code/number for the to-be-processed dataset
subject_code = 'sub-066'

# Set the path to the data folder
data_folder = server_folder + 'experiments/NEvent/exp_eeg_v1/' 
data_raw_file = os.path.join(data_folder,subject_code,
                             'eeg','raw',f'{subject_code}_NEvent-task.vhdr')

## Create EEG epochs from event markers 

In [5]:
def load_and_process_eeg_data(data_folder, subject_code):
    # Construct the file path
    fname = os.path.join(data_folder, subject_code, 'eeg', 'postproc', 
                         f'{subject_code}_filtered_annot_crop.fif')

    # Read the EEG data from the file
    raw = mne.io.read_raw_fif(fname)

    # fig = raw.plot(start=0, duration=60, n_channels=30, scalings=scalings)

    # Get the data channels
    data_channels = raw.ch_names

    # Return the raw data and the channel names
    return raw, data_channels

#call function
#raw, data_channels = load_and_process_eeg_data(data_folder, subject_code)

In [6]:
def read_event_markers (raw):
    # Read the event markers and display them. Confirm the 
    # codes correspond to the dictionary in the next cell
    events, tmp = mne.events_from_annotations(raw)

    # Count up the number of each event
    values, counts = np.unique(events[:,2], return_counts=True)
    print('\nEvent counts:',counts)
    #standard should be 480 480
    possible_events = {'Bit0/B  1': 'Tone',
                   'Bit1/B  1': 'Image',
                   'Bit2/B  1': 'Arrow',
                   'Bit3/B  1': 'Distance',
                   'Bit4/B  1': 'Order',
                   'Bit5/B  1': 'Source',
                   'Bit6/B  1': 'Start',
                   'New Segment/':'New Segment/'}

    event_id = {}
    for ikey in tmp.keys():
        event_id[possible_events[ikey]] = tmp[ikey]

    del tmp
    return events, counts, event_id

#call function
#events, event_counts, event_id = read_event_markers(raw)

In [7]:
def create_epochs(raw,events, event_id):
    
    tmin, tmax = -0.1, 1

    condition_label = 'Tone'
    # Create epochs
    epochs = mne.Epochs(raw, events, event_id=event_id[condition_label], 
                    tmin=tmin, tmax=tmax, baseline=(None,0))

    # Load the epoch data into an array
    #epoch_data = epochs.get_data(picks='Pz')
    epoch_data = epochs.get_data(picks=data_channels)

    # Convert the data into units of microvolts!
    epoch_data = epoch_data * 1e6
    return epoch_data, condition_label, epochs

#epoch_data, condition_label, epochs = create_epochs(raw,events, event_id)

## Load the behavioral data into a dataframe

In [8]:
def load_beh(server_folder, subject_code):
    beh_path = server_folder + 'experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_' + str(subject_code) + '.csv'
    print(beh_path)
    merged_file = pd.read_csv(beh_path)
    merged_file.head()
    print(merged_file.shape) #(480, 53) with practice trials ; (448,53) if without pratice trials 
    
    return merged_file,beh_path

#merged_file, beh_path= load_beh(server_folder, subject_code)
#print(beh_path)
# 484 rows in the dataframe --> B/NB tones

### CREATE EEG epochs for nb and b

In [12]:
def create_indiv_epochs(epochs, events, event_id, merged_file, epoch_data, condition_label):

    # Convert condition label to event ID and find corresponding epochs
    event_value = event_id[condition_label]
    condition_events = events[:, 2] == event_value

    # Identify which epochs were retained (not dropped due to annotation)
    retained_epochs = epochs.selection
    undropped_condition_events = np.isin(np.where(condition_events)[0], retained_epochs)

    # Filter the behavioral data to match the undropped, condition-specific epochs, AND pratice round
    b_epochs_indices = (merged_file['TrueBoundary.x'][undropped_condition_events] == 'B') & (merged_file['list_num'][undropped_condition_events] != 0)
    nb_epochs_indices = (merged_file['TrueBoundary.x'][undropped_condition_events] == 'NB') & (merged_file['list_num'][undropped_condition_events] != 0)


    # Select epochs based on behavioral indices
    b_epochs = epoch_data[b_epochs_indices, :, :]
    nb_epochs = epoch_data[nb_epochs_indices, :, :]

    # Print shapes of the epochs for boundary and non-boundary conditions
    #print(f"The shape of nb_epochs is {nb_epochs.shape}")
    #print(f"The shape of b_epochs is {b_epochs.shape}")

    # Averaging all B, NB, NB controlled trials
    b_ERP = np.nanmean(b_epochs, axis=0)
    nb_ERP = np.nanmean(nb_epochs, axis=0)

    # Print ERP shapes
    #print(nb_ERP.shape)
    #print(b_ERP.shape)

    return b_ERP, nb_ERP, b_epochs, nb_epochs

#call function
b_ERP, nb_ERP, b_epochs, nb_epochs= create_indiv_epochs(epochs, events, event_id, merged_file, epoch_data, condition_label)


The shape of nb_epochs is (332, 63, 1101)
The shape of b_epochs is (42, 63, 1101)
(63, 1101)
(63, 1101)


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_53282/1856373255.py:25: RuntimeWarning: Mean of empty slice
  b_ERP = np.nanmean(b_epochs, axis=0)
/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_53282/1856373255.py:26: RuntimeWarning: Mean of empty slice
  nb_ERP = np.nanmean(nb_epochs, axis=0)


### CREATE EEG epochs for Correct and Incorrect source (For Boundary trial only)

## Plotting for individual ERPs

In [10]:
#electrodes of interests (for graphing)
electrode_names = ['Cz', 'Pz', 'Fz', 'P3', 'P4', 'C4', 'C3', 'FC3']

def plot_indiv_erps(electrode_names, data_channels, b_ERP, nb_ERP, condition_label, data_folder, subject_code):
    # Create subplots
    fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20, 10)) 

    # Flatten the axs array for easy iteration
    axs = axs.flatten()

    for ax, electrode in zip(axs, electrode_names):
        electrode_idx = np.array(data_channels) == electrode

        # Plot ERP for Boundary and NonBoundary conditions
        ax.plot(b_ERP[electrode_idx,:].T) 
        ax.plot(nb_ERP[electrode_idx,:].T)
        ax.legend(['Boundary','NonBoundary'])
        ax.set_xlabel('Time Rel. Stimulus Onset (ms)'); 
        ax.set_ylabel('Voltage $\mu$V');
        ax.set_xticks([0,100,200,400,600,800,1000]); # Time point of the vector
        ax.set_xticklabels([-100,0,100,300,500,700,900]); # Actual time point
        ax.set_title(f'{condition_label}_{electrode}')

    # Remove unused subplots if the number of electrodes is less than the number of subplots
    for i in range(len(electrode_names), len(axs)):
        fig.delaxes(axs[i])

    # Adjust the space between the subplots
    plt.subplots_adjust(wspace=0.3, hspace=0.3)

    # Define the filename for the saved figure
    figure_fname = os.path.join(data_folder, subject_code, 'eeg', 'analysis', f'Without_practice_{subject_code}_{condition_label}_All_Electrodes.pdf')
    
    # Save the figure
    plt.savefig(figure_fname, bbox_inches='tight')
    plt.close(fig)  # Close the figure to free memory
    print(f'The figure for {subject_code} is saved')

    return figure_fname

#function call
#plot_indiv_erps(electrode_names, data_channels, b_ERP, nb_ERP, condition_label, data_folder, subject_code)


In [14]:

# isubj= subject_code
# raw, data_channels = load_and_process_eeg_data(data_folder, isubj)
# events, event_counts, event_id = read_event_markers(raw)
# epoch_data, condition_label, epochs = create_epochs(raw,events, event_id)
# merged_file, beh_path= load_beh(server_folder, isubj)
# b_ERP, nb_ERP, b_epochs, nb_epochs = create_indiv_epochs(epochs, events, event_id, merged_file, epoch_data, condition_label)
# plot_indiv_erps(electrode_names, data_channels, b_ERP, nb_ERP, condition_label, data_folder, isubj)


'\nisubj= subject_code\nraw, data_channels = load_and_process_eeg_data(data_folder, isubj)\nevents, event_counts, event_id = read_event_markers(raw)\nepoch_data, condition_label, epochs = create_epochs(raw,events, event_id)\nmerged_file, beh_path= load_beh(server_folder, isubj)\nb_ERP, nb_ERP, b_epochs, nb_epochs = create_indiv_epochs(epochs, events, event_id, merged_file, epoch_data, condition_label)\nplot_indiv_erps(electrode_names, data_channels, b_ERP, nb_ERP, condition_label, data_folder, isubj)\n'